In [1]:
import os
from tqdm import tqdm
tqdm.pandas(desc="my bar!")
from time import time
%load_ext jupyternotify

<IPython.core.display.Javascript object>

In [2]:
y_field = 'y:label'
# data_file_name = '../data/kddcup.data_10_percent.gz'
data_file_name = '../data/kddcup.data.gz'

In [3]:
features = ["duration: continuous.", "protocol_type: symbolic.", "service: symbolic.", "flag: symbolic.", "src_bytes: continuous.", "dst_bytes: continuous.", "land: symbolic.", "wrong_fragment: continuous.", "urgent: continuous.", "hot: continuous.", "num_failed_logins: continuous.", "logged_in: symbolic.", "num_compromised: continuous.", "root_shell: continuous.", "su_attempted: continuous.", "num_root: continuous.", "num_file_creations: continuous.", "num_shells: continuous.", "num_access_files: continuous.", "num_outbound_cmds: continuous.", "is_host_login: symbolic.", "is_guest_login: symbolic.", "count: continuous.", "srv_count: continuous.", "serror_rate: continuous.", "srv_serror_rate: continuous.", "rerror_rate: continuous.", "srv_rerror_rate: continuous.", "same_srv_rate: continuous.", "diff_srv_rate: continuous.", "srv_diff_host_rate: continuous.", "dst_host_count: continuous.", "dst_host_srv_count: continuous.", "dst_host_same_srv_rate: continuous.", "dst_host_diff_srv_rate: continuous.", "dst_host_same_src_port_rate: continuous.", "dst_host_srv_diff_host_rate: continuous.", "dst_host_serror_rate: continuous.", "dst_host_srv_serror_rate: continuous.", "dst_host_rerror_rate: continuous.", "dst_host_srv_rerror_rate: continuous.",'y:label']
features_0 = ["duration", "protocol_type", "service", "flag", "src_bytes", "dst_bytes", "land", "wrong_fragment", "urgent", "hot", "num_failed_logins", "logged_in", "num_compromised", "root_shell", "su_attempted", "num_root", "num_file_creations", "num_shells", "num_access_files", "num_outbound_cmds", "is_host_login", "is_guest_login", "count", "srv_count", "serror_rate", "srv_serror_rate", "rerror_rate", "srv_rerror_rate", "same_srv_rate", "diff_srv_rate", "srv_diff_host_rate", "dst_host_count", "dst_host_srv_count", "dst_host_same_srv_rate", "dst_host_diff_srv_rate", "dst_host_same_src_port_rate", "dst_host_srv_diff_host_rate", "dst_host_serror_rate", "dst_host_srv_serror_rate", "dst_host_rerror_rate", "dst_host_srv_rerror_rate","y"]
df = pd.read_csv(data_file_name, names=features)
print(df.shape)
df.head()

(4898431, 42)


,duration: continuous.,protocol_type: symbolic.,service: symbolic.,flag: symbolic.,src_bytes: continuous.,dst_bytes: continuous.,land: symbolic.,wrong_fragment: continuous.,urgent: continuous.,hot: continuous.,...,dst_host_srv_count: continuous.,dst_host_same_srv_rate: continuous.,dst_host_diff_srv_rate: continuous.,dst_host_same_src_port_rate: continuous.,dst_host_srv_diff_host_rate: continuous.,dst_host_serror_rate: continuous.,dst_host_srv_serror_rate: continuous.,dst_host_rerror_rate: continuous.,dst_host_srv_rerror_rate: continuous.,y:label
0,0,tcp,http,SF,215,45076,0,0,0,0,...,0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,normal.
1,0,tcp,http,SF,162,4528,0,0,0,0,...,1,1.0,0.0,1.00,0.0,0.0,0.0,0.0,0.0,normal.
2,0,tcp,http,SF,236,1228,0,0,0,0,...,2,1.0,0.0,0.50,0.0,0.0,0.0,0.0,0.0,normal.
3,0,tcp,http,SF,233,2032,0,0,0,0,...,3,1.0,0.0,0.33,0.0,0.0,0.0,0.0,0.0,normal.
4,0,tcp,http,SF,239,486,0,0,0,0,...,4,1.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,normal.


In [4]:
n_classes = df[y_field].nunique()
n_classes

23

In [5]:
features_continuous = [x for x in features if 'continuous' in x]
len(features_continuous)

34

In [10]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedShuffleSplit,StratifiedKFold,KFold
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix,precision_score,classification_report

In [ ]:
# df[features_continuous] = MinMaxScaler().fit_transform(df[features_continuous])
# df[features_continuous].describe()

In [14]:
cls = SVC()
cls = RandomForestClassifier(n_jobs=10)
# cls = LogisticRegression(n_jobs=10)
simple_pipeline = Pipeline(
[
    ('scaler',MinMaxScaler()),
    ('cls',cls)
],
)

In [15]:
%%notify
t = time()
cv = StratifiedKFold(n_splits=2, shuffle=True)
# cv = StratifiedShuffleSplit(n_splits=3,test_size=0.1)
cms = np.ndarray((cv.n_splits, n_classes * n_classes), int)
for i, (train, test) in zip(range(cv.n_splits),cv.split(df,df[y_field])):
    simple_pipeline.fit(df.iloc[train][features_continuous],df.iloc[train][y_field])
    y_pred = simple_pipeline.predict(df.iloc[test][features_continuous])
    cm = confusion_matrix(df.iloc[test][y_field], y_pred)
    print(cm)
    print("p_micro:{},p_macro:{}".format(precision_score(df.iloc[test][y_field],y_pred,average='micro'),
                                        precision_score(df.iloc[test][y_field],y_pred,average='macro')))
    print(classification_report(df.iloc[test][y_field],y_pred))
    cms[i] = cm.flatten()
avg_cms = cms.mean(axis=0).reshape((n_classes, n_classes)).astype(int)
print('average confusion matrix:\n{}'.format(avg_cms))
print('estimation finished. Elapsed: {}'.format(time() - t))

[[   1095       0       0       0       0       0       0       0       0
        0       0       7       0       0       0       0       0       0
        0       0       0       0       0]
 [      0       5       0       0       0       0       0       0       0
        0       0      10       0       0       0       0       0       0
        0       0       0       0       0]
 [      0       0       1       0       0       0       0       0       0
        0       0       3       0       0       0       0       0       0
        0       0       0       0       0]
 [      0       0       0      24       0       0       0       0       0
        0       0       3       0       0       0       0       0       0
        0       0       0       0       0]
 [      0       0       0       0       4       0       0       0       0
        0       0       2       0       0       0       0       0       0
        0       0       0       0       0]
 [      0       0       0       0       0    

D:\MyProgramFiles\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


p_micro:0.9997799300267309,p_macro:0.7757871311786412


D:\MyProgramFiles\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


                  precision    recall  f1-score   support

           back.       1.00      0.99      1.00      1102
buffer_overflow.       0.83      0.33      0.48        15
      ftp_write.       1.00      0.25      0.40         4
   guess_passwd.       1.00      0.89      0.94        27
           imap.       1.00      0.67      0.80         6
        ipsweep.       0.99      0.99      0.99      6241
           land.       0.58      0.64      0.61        11
     loadmodule.       1.00      0.40      0.57         5
       multihop.       0.50      0.25      0.33         4
        neptune.       1.00      1.00      1.00    536009
           nmap.       0.99      0.94      0.96      1158
         normal.       1.00      1.00      1.00    486391
           perl.       0.00      0.00      0.00         2
            phf.       0.00      0.00      0.00         2
            pod.       0.97      0.98      0.98       132
      portsweep.       1.00      1.00      1.00      5207
        rootk

D:\MyProgramFiles\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


p_micro:0.9997733963196296,p_macro:0.7938338009007756


D:\MyProgramFiles\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


                  precision    recall  f1-score   support

           back.       1.00      1.00      1.00      1101
buffer_overflow.       0.89      0.53      0.67        15
      ftp_write.       1.00      0.25      0.40         4
   guess_passwd.       1.00      1.00      1.00        26
           imap.       1.00      0.50      0.67         6
        ipsweep.       0.99      0.99      0.99      6240
           land.       0.42      0.50      0.45        10
     loadmodule.       1.00      0.50      0.67         4
       multihop.       0.00      0.00      0.00         3
        neptune.       1.00      1.00      1.00    536008
           nmap.       0.98      0.95      0.96      1158
         normal.       1.00      1.00      1.00    486390
           perl.       0.00      0.00      0.00         1
            phf.       1.00      0.50      0.67         2
            pod.       1.00      0.96      0.98       132
      portsweep.       1.00      1.00      1.00      5206
        rootk

<IPython.core.display.Javascript object>